<a href="https://colab.research.google.com/github/voldo-roma/nosi_den/blob/main/numpy_speedup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PIL code to paste an image onto another:

In [1]:
def blend_mask_onto_image_positioned(original_img, rle_mask, highlight_color, scale_factor, x, y, opacity=0.3):
  highlight_color = color_name_to_rgb(highlight_color)
  # 1. Decode the RLE mask
  mask_array = coco_mask.decode(rle_mask)
  mask_image = Image.fromarray((mask_array * 255).astype(np.uint8))

  # 2. Scale the mask
  new_size = (int(mask_image.width * scale_factor), int(mask_image.height * scale_factor))
  #print(f"old size: {mask_image.width} new size: {mask_image.width * scale_factor}")
  scaled_mask = mask_image.resize(new_size, Image.LANCZOS)

  # Position the scaled mask on a blank canvas of the same size as the original image at (x, y)
  full_size_mask = Image.new("L", original_img.size)
  full_size_mask.paste(scaled_mask, (x, y))


  highlight_img = Image.new("RGBA", original_img.size, highlight_color + (0,))  # Start with 0 opacity
  overlay = Image.new("RGBA", original_img.size, highlight_color + (int(255 * opacity),))  # Desired opacity
  highlight_img = Image.composite(overlay, highlight_img, full_size_mask)

  # 4. Blend the images
  blended_img = Image.alpha_composite(original_img.convert('RGBA'), highlight_img).convert('RGB')


  return blended_img

Numpy code to do the same:

In [ ]:
def blend_mask_onto_image_positioned(original_array, rle_mask, highlight_color, scale_factor, x, y, opacity=0.3):

      # 1. Decode the RLE mask
      mask_array = coco_mask.decode(rle_mask)

      new_shape = (int(mask_array.shape[0] * scale_factor), int(mask_array.shape[1] * scale_factor))
      # 2. scale the mask
      scaled_mask = resize(mask_array, new_shape, order=1, mode='reflect', cval=0, clip=True, preserve_range=True)

      # 3. Position the scaled mask on a blank canvas of the same size as the original image at (x, y)
      full_size_mask = np.zeros(original_array.shape[:2], dtype=scaled_mask.dtype)
      full_size_mask[y:y+scaled_mask.shape[0], x:x+scaled_mask.shape[1]] = scaled_mask

      # Determine the regions where blending is necessary directly from the scaled mask
      blend_rows, blend_cols = np.where(scaled_mask)

      # Adjust for the positioning on the original image
      blend_rows += y
      blend_cols += x


      # Compute the alpha values for these regions
      alpha_values = full_size_mask[blend_rows, blend_cols] * opacity

      # Compute the highlight color for these regions
      highlight_color_rgb = np.array(color_name_to_rgb(highlight_color))

      # Blend the original and highlight colors for these regions
      blended_array = original_array.copy()
      blended_array[blend_rows, blend_cols] = (original_array[blend_rows, blend_cols] * (1 - alpha_values[:, np.newaxis]) +
                                                highlight_color_rgb * alpha_values[:, np.newaxis]).astype(np.uint8)

      return blended_array